#  RAG - Retrieval-Augmented Generation

### 1. Document Loading

* Use **Document Loaders** in LangChain.
* Examples: `PyPDFLoader`, `TextLoader`, `UnstructuredFileLoader`.
* Purpose: Bring external data (PDFs, text files, web pages) into LangChain.

---

### 2. Document Splitting

* Use **Text Splitters** like `CharacterTextSplitter` or `RecursiveCharacterTextSplitter`.
* Purpose: Break large documents into **small chunks** (e.g., 500–1000 tokens).
* Why: LLMs work better on smaller pieces of text.

---

### 3. Document Embedding

* Use **Embeddings models** (e.g., `OpenAIEmbeddings`, `HuggingFaceEmbeddings`).
* Each chunk → vector (list of numbers).
* **Cosine Similarity** is used to measure how close two vectors (chunks) are.

  * Example: Question vector vs stored chunk vectors → find the most similar ones.

---

### 4. Document Storing

* Store embeddings in a **Vector Store** (like FAISS, Chroma, Pinecone).
* Purpose: Efficient search & retrieval of similar chunks later.

---

### 5. Retrieval + Generation (RAG)

* When user asks a question:

  * Convert question → embedding.
  * Retrieve top-k **relevant and diverse** chunks from Vector Store (using cosine similarity).
  * Send question + retrieved context to LLM via a **RetrievalQA chain**.
  * LLM generates the final answer.

---

So in LangChain terms:
**Loader → Splitter → Embeddings → Vector Store → RetrievalQA Chain**

---

# Indexing: Document Loading with PyPDF Loader

In [45]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_text_splitters.markdown import MarkdownHeaderTextSplitter
import copy

In [6]:
loader_pdf = PyPDFLoader(r"D:\AI-ML\Langchain\Files\Introduction_to_Data_and_Data_Science.pdf")

pages_pdf = loader_pdf.load()

pages_pdf

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2023-11-09T10:16:34+02:00', 'author': 'Hristina  Hristova', 'moddate': '2023-11-09T10:16:34+02:00', 'source': 'D:\\AI-ML\\Langchain\\Files\\Introduction_to_Data_and_Data_Science.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='Analysis vs Analytics \nAlright! So… \nLet’s discuss the not-so-obvious differences \nbetween the terms analysis and analytics. \nDue to the similarity of the words, some people \nbelieve they share the same meaning, and thus \nuse them interchangeably. Technically, this \nisn’t correct. There is, in fact, a distinct \ndifference between the two. And the reason \nfor one often being used instead of the other \nis the lack of a transparent understanding \nof both. \nSo, let’s clear this up, shall we? \nFirst, we will start with analysis. \nConsider the following… \nYou have a huge dataset containing data of \nvar

In [7]:
pages_pdf_cut = copy.deepcopy(pages_pdf)

### 1. `copy.deepcopy()`

* Comes from Python’s built-in `copy` module.
* `deepcopy()` creates a **new independent copy** of the object *and all objects inside it*.
* This means changes in the copy will **not affect** the original.

---

### 2. Why use `deepcopy` here?

* If you just did:

  ```python
  pages_pdf_cut = pages_pdf
  ```

  → Both variables point to the **same object** in memory. Editing one edits the other.

* With `deepcopy`:

  ```python
  pages_pdf_cut = copy.deepcopy(pages_pdf)
  ```

  → `pages_pdf_cut` is a **completely separate clone**. You can modify it (cut pages, clean text, etc.) without touching `pages_pdf`.

---

In short:
This line makes a **full independent duplicate** of `pages_pdf`, so you can safely work on `pages_pdf_cut` without messing up the original data.


In [10]:
for i in pages_pdf_cut:
    i.page_content = ' '.join(i.page_content.split()) # Removes \n

In [11]:
pages_pdf[0].page_content, pages_pdf_cut[0].page_content

('Analysis vs Analytics \nAlright! So… \nLet’s discuss the not-so-obvious differences \nbetween the terms analysis and analytics. \nDue to the similarity of the words, some people \nbelieve they share the same meaning, and thus \nuse them interchangeably. Technically, this \nisn’t correct. There is, in fact, a distinct \ndifference between the two. And the reason \nfor one often being used instead of the other \nis the lack of a transparent understanding \nof both. \nSo, let’s clear this up, shall we? \nFirst, we will start with analysis. \nConsider the following… \nYou have a huge dataset containing data of \nvarious types. Instead of tackling the entire \ndataset and running the risk of becoming overwhelmed, \nyou separate it into easier to digest chunks \nand study them individually and examine how \nthey relate to other parts. And that’s analysis \nin a nutshell. \nOne important thing to remember, however, \nis that you perform analyses on things that \nhave already happened in the

# Indexing: Document loading with Docx2txtLoader

* PDF loader: granular, page-by-page, rich metadata.
* DOCX loader: all-in-one chunk, minimal metadata.

In [52]:
loader_docx = Docx2txtLoader(r"D:\AI-ML\Langchain\Files\Introduction_to_Data_and_Data_Science.docx")

pages_docx = loader_docx.load()

pages_docx

[Document(metadata={'source': 'D:\\AI-ML\\Langchain\\Files\\Introduction_to_Data_and_Data_Science.docx'}, page_content="Analysis vs Analytics\n\nAlright! So…\nLet’s discuss the not-so-obvious differences\nbetween the terms analysis and analytics.\nDue to the similarity of the words, some people\nbelieve they share the same meaning, and thus\nuse them interchangeably. Technically, this\nisn’t correct. There is, in fact, a distinct\ndifference between the two. And the reason\nfor one often being used instead of the other\nis the lack of a transparent understanding\nof both.\nSo, let’s clear this up, shall we?\nFirst, we will start with analysis.\nConsider the following…\nYou have a huge dataset containing data of\nvarious types. Instead of tackling the entire\ndataset and running the risk of becoming overwhelmed,\nyou separate it into easier to digest chunks\nand study them individually and examine how\nthey relate to other parts. And that’s analysis\nin a nutshell.\nOne important thing 

# Indexing: Document splitting with character text splitter (Theory)

In [62]:
loader = Docx2txtLoader(r"D:\AI-ML\Langchain\Files\Introduction_to_Data_and_Data_Science.docx")
pages = loader.load()

pages[0].page_content = ' '.join(pages[0].page_content.split())

In [63]:
len(pages[0].page_content)

8259

In [58]:
char_splitter = CharacterTextSplitter(separator = ".", # Split for full stops
                                      chunk_size = 500, # Number of Characters
                                      chunk_overlap = 50)

In [25]:
pages_char_split = char_splitter.split_documents(pages)

In [26]:
len(pages_char_split)

21

In [37]:
pages_char_split[0].page_content, pages_char_split[1].page_content

('Analysis vs Analytics Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both',
 'So, let’s clear this up, shall we? First, we will start with analysis. Consider the following… You have a huge dataset containing data of various types. Instead of tackling the entire dataset and running the risk of becoming overwhelmed, you separate it into easier to digest chunks and study them individually and examine how they relate to other parts. And that’s analysis in a nutshell')

In [36]:
len(pages_char_split[1].page_content)

388

In [42]:
for content in pages_char_split:
    content.page_content = content.page_content + "."

# Document splitting with Markdown header text splitter

In [72]:
loader = Docx2txtLoader(r"D:\AI-ML\Langchain\Files\Introduction_to_Data_and_Data_Science_2.docx")
pages = loader.load()

In [73]:
pages

[Document(metadata={'source': 'D:\\AI-ML\\Langchain\\Files\\Introduction_to_Data_and_Data_Science_2.docx'}, page_content="# Intro to Langchain\n\n## Analysis vs Analytics\n\nAlright! So…\nLet’s discuss the not-so-obvious differences\nbetween the terms analysis and analytics.\nDue to the similarity of the words, some people\nbelieve they share the same meaning, and thus\nuse them interchangeably. Technically, this\nisn’t correct. There is, in fact, a distinct\ndifference between the two. And the reason\nfor one often being used instead of the other\nis the lack of a transparent understanding\nof both.\nSo, let’s clear this up, shall we?\nFirst, we will start with analysis.\nConsider the following…\nYou have a huge dataset containing data of\nvarious types. Instead of tackling the entire\ndataset and running the risk of becoming overwhelmed,\nyou separate it into easier to digest chunks\nand study them individually and examine how\nthey relate to other parts. And that’s analysis\nin a nu

In [74]:
md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on = [("#", "Course Title"), 
                                                                ("##", "Lecture Title")])

In [75]:
pages_md_split = md_splitter.split_text(pages[0].page_content)

pages_md_split

[Document(metadata={'Course Title': 'Intro to Langchain', 'Lecture Title': 'Analysis vs Analytics'}, page_content="Alright! So…\nLet’s discuss the not-so-obvious differences\nbetween the terms analysis and analytics.\nDue to the similarity of the words, some people\nbelieve they share the same meaning, and thus\nuse them interchangeably. Technically, this\nisn’t correct. There is, in fact, a distinct\ndifference between the two. And the reason\nfor one often being used instead of the other\nis the lack of a transparent understanding\nof both.\nSo, let’s clear this up, shall we?\nFirst, we will start with analysis.\nConsider the following…\nYou have a huge dataset containing data of\nvarious types. Instead of tackling the entire\ndataset and running the risk of becoming overwhelmed,\nyou separate it into easier to digest chunks\nand study them individually and examine how\nthey relate to other parts. And that’s analysis\nin a nutshell.\nOne important thing to remember, however,\nis that

In [51]:
pages[0].page_content

"# Intro to Langchain ## Analysis vs Analytics Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both. So, let’s clear this up, shall we? First, we will start with analysis. Consider the following… You have a huge dataset containing data of various types. Instead of tackling the entire dataset and running the risk of becoming overwhelmed, you separate it into easier to digest chunks and study them individually and examine how they relate to other parts. And that’s analysis in a nutshell. One important thing to remember, however, is that you perform analyses on things that have already happened in the past. Such as using an a